In [12]:
import numpy             as np
import gvar              as gv
import matplotlib.pyplot as plt
import lsqfit

In [13]:
from b2heavy import FnalHISQMetadata

In [14]:
from b2heavy.TwoPointFunctions.utils     import correlation_diagnostics
from b2heavy.TwoPointFunctions.types2pts import CorrelatorIO, plot_effective_coeffs
from b2heavy.TwoPointFunctions.fitter    import StagFitter

In [15]:
from b2heavy.ThreePointFunctions.types3pts  import Ratio, RatioIO
from b2heavy.ThreePointFunctions.fitter3pts import RatioFitter

Select a case for which we want to calculate the recoil parameter

In [16]:
ENSEMBLE = 'Coarse-1'
DATA_DIR = '/Users/pietro/code/data_analysis/BtoD/Alex/'

BINSIZE  = {
    'MediumCoarse': 13,
    'Coarse-1': 11
} 

COV_SPECS = dict(
    diag   = False,
    block  = False,
    scale  = True,
    shrink = True,
    cutsvd = 0.01
)

In [17]:
mdata = FnalHISQMetadata.params(ENSEMBLE)
LVOL   = mdata['L']
alphas = mdata['alphaS'] 

## Two-pts functions

In [18]:
MESON      = 'Dst'
SMSLIST  = ['1S-1S','d-d','d-1S']
MOM_LIST   = ['000','100','200','300','110','211']
TRANGE_EFF = {
    'Coarse-1': {
        '000': (13,20),
        '100': (13,20),
        '200': (13,20),
        '300': (10,17),
        '110': (11,19),
        '211': (11,19)
    }
}
TRANGE     = {
    'Coarse-1': {
        '000': (5,26),
        '100': (5,23),
        '200': (5,21),
        '300': (5,17),
        '110': (5,23),
        '211': (5,19)
    }
}

In [19]:
# fits = {}
# for mom in MOM_LIST:
#     print(f'{ mom = :-^20}')

#     io   = CorrelatorIO(ENSEMBLE,MESON,mom,PathToDataDir=DATA_DIR)
#     stag = StagFitter(
#         io       = io,
#         jkBin    = BINSIZE[ENSEMBLE],
#         smearing = SMSLIST
#     )

#     effm,effa = stag.meff(
#         TRANGE_EFF[ENSEMBLE][mom],
#         **COV_SPECS
#     )

#     fits[mom] = stag.fit(
#         Nstates = 3,
#         trange  = TRANGE[ENSEMBLE][mom],
#         **COV_SPECS
#     )

In [20]:
if False:
    params = {}
    for mom in MOM_LIST:
        print(f'{ mom = :-^20}')

        io   = CorrelatorIO(ENSEMBLE,MESON,mom,PathToDataDir=DATA_DIR)
        stag = StagFitter(
            io       = io,
            jkBin    = BINSIZE[ENSEMBLE],
            smearing = SMSLIST
        )

        effm,effa = stag.meff(
            TRANGE_EFF[ENSEMBLE][mom],
            **COV_SPECS
        )

        jk = stag.fit(
            Nstates = 3,
            trange  = TRANGE[ENSEMBLE][mom],
            jkfit   = True,
            **COV_SPECS
        )

        params[mom] = jk

    for m in params:
        np.save(f'jkfit_{m}.npy',np.asarray(params[m]))

In [34]:
params = {mom: np.load(f'jkfit_{mom}.npy') for mom in MOM_LIST}

In [22]:
io   = CorrelatorIO(ENSEMBLE,MESON,'100',PathToDataDir=DATA_DIR)
stag = StagFitter(
    io       = io,
    jkBin    = BINSIZE[ENSEMBLE],
    smearing = SMSLIST
)

effm,effa = stag.meff(
    TRANGE_EFF[ENSEMBLE]['100'],
    **COV_SPECS
)

jk = stag.fit(
    Nstates = 3,
    trange  = TRANGE[ENSEMBLE]['100'],
    jkfit   = True,
    **COV_SPECS
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
100%|██████████| 90/90 [00:40<00:00,  2.24it/s]


In [33]:
m0   = params['000'][:,0]
E0   = jk['E'][:,0]
Zpar = jk['Z_1S_Par'][:,0]
Zbot = jk['Z_1S_Bot'][:,0]

## Three-pts functions

In [29]:
RATIO  = 'R0'
MOM    = '100'
SMLIST = ['1S']

Nstates = 1
trange  = (3,11)

In [30]:
energy = {mom: params[mom][:,0] for mom in params}

In [31]:
m0 = gv.gvar(energy['000'].mean(),energy['000'].std()*np.sqrt(len(energy['000']-1)))
E0 = gv.gvar(energy[MOM].mean(),energy[MOM].std()*np.sqrt(len(energy[MOM]-1)))

E0-m0

0.0160(83)

Initialize ratio objects

In [32]:
io = RatioIO(ENSEMBLE,RATIO,MOM,PathToDataDir=DATA_DIR)
data = io.ReadRatio(
    jkBin= 11,
    E0   = E0,
    m0   = m0,
    sms  = ['1S']
)


Exception: For R0 ratio, Z_1S_par and Z_1S_bot must be provided

In [ ]:
ratio